<a href="https://colab.research.google.com/github/sccn/sound2meg/blob/main/Spatial_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import os
from torch.autograd import Variable
import math
import torch.nn as nn
from torch.nn.parameter import Parameter
import torch.optim as optim

#text

In [ ]:
#import scipy.io
#data = scipy.io.loadmat('/content/drive/MyDrive/data.mat')

In [ ]:
#x = data['data']
#x = x[:, 1:3600:10, :]
#print(x.shape)

(273, 360, 7)


In [ ]:
def cos_vector(k):
  cos_v = torch.zeros(N, K)
  for l in range(K):
    cos_v[:,l] = torch.cos(2*math.pi*(k*x[:, 0]+l*y[:, 0]))
  return cos_v
def sin_vector(k):
  sin_v = torch.zeros(N, K)
  for l in range(K):
    sin_v[:,l] = torch.sin(2*math.pi*(k*x[:, 0]+l*y[:, 0]))
  return sin_v

In [ ]:
def SpatialAttentionFunc(N, X, x, y, z_R, z_I, K):
  a = torch.randn(N, N)
  for j in range(N):
    cos_sum = torch.zeros(N)
    sin_sum = torch.zeros(N)
    for k in range(K):
      z_r = z_R[j, k, :].unsqueeze(0)
      z_i = z_I[j, k, :].unsqueeze(0)
      cos_k = torch.transpose(cos_vector(k), 0, 1)
      sin_k = torch.transpose(sin_vector(k), 0, 1)
      cos_sum = cos_sum + torch.mm(z_r, cos_k)
      sin_sum = sin_sum + torch.mm(z_r, cos_k)
    a[j, :] = cos_sum + sin_sum 
  SA = torch.randn(N, 360)
  for j in range(N):
    SA[j] = torch.mm(nn.functional.softmax(a[j,:].unsqueeze(0), -1), X)
    exp1 = torch.mm(torch.exp(a[j, :]/100).unsqueeze(0), X)
    exp2 = torch.sum(torch.exp(a[j, :]/100))
    #SA[j] = exp1/exp2
  return SA

In [ ]:
x = torch.randn(N, 1)
y = torch.randn(N, 1)

NameError: ignored

In [ ]:
K = 32
z_R = Variable(torch.zeros(N, K, K), requires_grad = True)
z_I = Variable(torch.zeros(N, K, K), requires_grad = True)

In [ ]:
class SubjectLayer(nn.Module):
  def __init__(self, x, subject_id):
    super(SubjectLayer, self).__init__()
    self.x = x
    self.s_idx = subject_id - 2002
    self.conv = nn.Conv2d(270, 270, 1, padding = 'same')
  def forward(self):
    #x = torch.transpose(x, 0, 1)
    x = self.x[self.s_idx]
    x = self.conv(x)
    return x

In [ ]:
class SpatialAttention(nn.Module):
  def __init__(self, C, K):
    super(SpatialAttention, self).__init__()
    self.C = C
    self.K = K
    self.z_R = Parameter(torch.randn(N, K, K))
    self.z_I = Parameter(torch.randn(N, K, K))
    self.z_R.requiresGrad = True
    self.z_I.requiresGrad = True
  def forward(self, X, x, y):
    N = X.size()[0]
    SA = torch.zeros(N, 270, 360)
    for i in range(N):
      SA[i] = SpatialAttentionFunc(self.C, X[i], x, y, self.z_R, self.z_I, self.K)
    return SA

In [ ]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.glu = nn.GLU() # Don't need this custom glu variable
    self.SA = SpatialAttention(270, 32)
  def Conv(self,in_channels, out_channels, dilation, filter, padding): # should not have a wrapper for a same one-liner
    return nn.Conv2d(in_channels, out_channels, (filter, 1), dilation = dilation, padding = (padding, 0))
  def forward(self, x, x_coord, y_coord, F):
    x = self.SA(x, x_coord, y_coord).unsqueeze(0)
    print(x.shape)
    x = torch.permute(x, (1, 2, 3, 0)) # subject attention?
    for k in range(1,6):
      p = pow(2,(2*k)%5)
      q = pow(2,(2*k+1)%5)
      if k == 1:
        x = nn.Conv2d(270, 320, (3, 1), dilation = 1, padding = (1, 0))(x)
        x = nn.BatchNorm2d(320)(x)
        x = nn.GELU()(x)
        x = nn.Conv2d(320, 320, (3, 1), dilation = 1, padding = (1, 0))(x)
        x = nn.BatchNorm2d(320)(x)
        x = nn.GELU()(x)
        x = nn.Conv2d(320, 640, (3, 1), dilation = 2, padding = (2, 0))(x)
        #x = self.Conv(270, 320, 1, 3, 1)(x) # missing batch norm, and GELU
        #x = self.Conv(320, 320, 1, 3, 1)(x) # missing batch norm, and GELU
        #x = self.Conv(320, 640, 2, 3, 2)(x)
        x = torch.transpose(x, 3, 1)
        x = self.glu(x)
        x = torch.transpose(x, 3, 1)
      else:
        x1 = nn.Conv2d(320, 320, (3, 1), dilation = p, padding = (p, 0))(x)
        x1 = nn.BatchNorm2d(320)(x1)
        x1 = nn.GELU()(x1)
        x2 = x + x1
        x3 = nn.Conv2d(320, 320, (3, 1), dilation = q, padding = (q, 0))(x2)
        x3 = nn.BatchNorm2d(320)(x3)
        x3 = nn.GELU()(x3)
        x4 = x2 + x2
        x_out = nn.Conv2d(320, 640, (3, 1), dilation = 2, padding = (2, 0))(x4)
        #x1 = self.Conv(320, 320, p, 3, p)(x)
        #x2 = x + x1
        #x3 = self.Conv(320, 320, q, 3, q)(x2)
        #x4 = x2 + x3
        #x_out = self.Conv(320, 640, 2, 3, 2)(x4)
        x_out = torch.transpose(x_out, 3, 1)
        x_out = self.glu(x_out)
        x_out = torch.transpose(x_out, 3, 1)
    x_out = self.Conv(320, 640, 1, 1, 0)(x_out)
    x_out = nn.GELU()(x_out)
    x_out = nn.Conv2d(640, F, (1, 1))(x_out)
    # missing GELU
    # missing 1 more conv
    return x_out

In [ ]:
X = torch.randn(2, 270, 360)
Y = torch.randn(2, 640, 360)

In [ ]:
def CLIP_loss(Z, Y):
  N = Y.size(dim = 0)
  inner_product = torch.zeros(N, N)
  log_softmax = torch.zeros(N)
  for i in range(N):
    for j in range(N):
      Z_row = torch.reshape(Z[i], (-1,))
      Y_row = torch.reshape(Y[j], (-1,))
      inner_product[i, j] = torch.dot(Z_row, Y_row)/100
  for i in range(N):
    inn = inner_product[i, :]
    log_softmax[i] = torch.log(nn.functional.softmax(inn, -1))[i]
  return sum(-1*log_softmax)

In [ ]:
for i in range(10):
  net = Net()
  optimizer = optim.SGD(net.parameters(), lr=0.003, momentum=0.9)
  Z = net(X, x, y, 640)
  Z = Z[:, :, :, 0]
  loss = CLIP_loss(Z, Y)
  optimizer.zero_grad()
  loss.backward()
  print(loss)
  optimizer.step()

<ipython-input-19-5ae2ff60a88b>:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  SA[j] = torch.mm(nn.functional.softmax(a[j,:].unsqueeze(0)), X)


tensor(1.8136, grad_fn=<AddBackward0>)
tensor(0.9172, grad_fn=<AddBackward0>)
tensor(1.4450, grad_fn=<AddBackward0>)
tensor(1.2962, grad_fn=<AddBackward0>)
tensor(1.4740, grad_fn=<AddBackward0>)
tensor(0.9817, grad_fn=<AddBackward0>)


KeyboardInterrupt: ignored